In [1]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
from scipy.sparse.linalg import spsolve

In [2]:
# #This is the input for hand
# v, f = igl.read_triangle_mesh('data/hand.off')
# labels = np.load('data/hand.label.npy').astype(int)
# v -= v.min(axis=0)
# v /= v.max()

In [3]:
#This is the input for woody, try with woody first, 2D is faster
v, f = igl.read_triangle_mesh('data/woody-hi.off')
labels = np.load('data/woody-hi.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

## Step 1-1: select and deform the handle regions, done by Prof

In [4]:
handle_vertex_positions = v.copy() #This is the first step to select and deform the handle regions
v_smooth = handle_vertex_positions.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [5]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [6]:
# ## Widget UI
# ## Original Plot for hand.off and plot after smoothing

# p = mp.plot(handle_vertex_positions, f, c=labels)
# iw.interact(pos_f,
#             **widgets_wrapper())

### Step 1-1-Plot: The initial plot for woody

In [10]:
## Widget UI
## Plot for the woody-hi.off

p = mp.plot(handle_vertex_positions, f, c=labels, shading = {"wireframe": True, "flat": False})
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4, 5), value=1), FloatSlider(value=0.0, desc…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

## Step 1-2: Removal of high-frequency details
### What I need to do:
1. express Lw, M-1
2. use the similar solution in assign4 to add constraints

In [8]:
import scipy.sparse as sp
Lw = igl.cotmatrix(v, f)
M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
M_inverse = sp.diags(1 / M.data)

A = Lw @ M_inverse @ Lw
Aff = A[labels==0, :][:, labels==0]
Afc = A[labels==0, :][:, labels>0]
xc = v_smooth[labels>0, :]
x = sp.linalg.spsolve(Aff, -Afc @ xc)
v_smooth[labels==0] = x
p = mp.plot(v_smooth, f, c=labels, shading = {"wireframe": True, "flat": False})
# p = mp.plot(v_smooth, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

## Step 2: Deforming the smooth mesh
1. di = vis - viB (B is the smoothed one) so di is just the same shape with v
2. rotate the the details di with mesh B  
    2.1 Calculate normal ni (for surface B)  
    2.2 Project all neighboring vertices to the tangent plane (perpendicular to ni)  
    2.3 Find neighbor j* for which projected edge (i, j) is longest. Normalize this edge vector and call it xi.  
    2.4 Construct yi using the cross product, completing orthonormal frame (xi, yi, ni) 
3. di = dixxi + diyyi + dinni  inner products.

In [9]:
#B'
xc_2 = handle_vertex_positions[labels>0, :]
x_2 = sp.linalg.spsolve(Aff, -Afc @ xc_2)
v_smooth_2 = handle_vertex_positions
v_smooth_2[labels==0] = x_2
p_2 = mp.plot(v_smooth_2, f, c=labels, shading = {"wireframe": True, "flat": False})
# p_2 = mp.plot(v_smooth_2, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

In [10]:
def construct_mesh_details(v_sm):
    d = v - v_sm
    # print(d.shape, v.shape)
    normals = igl.per_vertex_normals(v_sm, f)
    neighs = igl.adjacency_list(f)
    j_longest = np.zeros(v.shape[0], dtype=np.int16)
    di = np.zeros_like(v)

    for i in range(v.shape[0]):
        ni = normals[i]
        i_j = v_sm[neighs[i]] - v_sm[i]
        projected_edges = i_j - i_j @ ni[:, None] * ni
        neighs_index = np.argmax(np.linalg.norm(projected_edges, axis=1))
        j_longest[i] = neighs[i][neighs_index]
        xi = projected_edges[neighs_index]
        xi /= np.linalg.norm(xi)
        yi = np.cross(ni, xi)
        di[i] = np.array([d[i] @ xi, d[i] @ yi, d[i] @ ni])
    return di, j_longest


In [11]:
#caculate for mesh B
di_B, j_longest_B = construct_mesh_details(v_smooth)
#caculate for mesh B'
di_B_2, j_longest_B_2 = construct_mesh_details(v_smooth_2)

# Step 3: Transferring high-frequency details to the deformed surface  
1. The unit vertex normal  
2. The normalized projection of one of 's outgoing edges onto the tangent plane defined by the vertex normal. A stable choice is the edge whose projection onto the tangent plane is longest.  
3. The cross-product between (1) and (2)

In [12]:
# v_detailed_2 = v_smooth_2.copy()
# ni= igl.per_vertex_normals(v_smooth_2, f)
# print(v_smooth_2.shape)
# print((ni*(np.sum(v_smooth_2 * ni, axis=1).reshape(-1, 1))).shape)

In [13]:
def transfer_details(v_sm, di, j_longest):
    di_new = np.zeros_like(di)
    v_detailed = v_sm.copy()
    ni = igl.per_vertex_normals(v_sm, f)
    edges = v_sm[j_longest] - v_sm
    xi = np.zeros_like(ni)
    xi = edges - ni*(np.sum(edges * ni, axis=1).reshape(-1, 1))
    xi /= np.linalg.norm(xi).reshape(-1, 1)
    yi = np.cross(ni, xi)
    for i in range(v.shape[0]):
#         di_new[i] = np.array([di[i] @ xi, di[i] @ yi, di[i] @ ni])
        di_new[i] = di[i] @ np.vstack((xi[i], yi[i], ni[i]))
    v_detailed += di_new    
    return v_detailed

In [14]:
# v_detailed_2 = transfer_details(v_smooth_2,di_B_2,j_longest_B_2)
# p_3 = mp.plot(v_detailed_2, f, c=labels)
# iw.interact(pos_f,
#             **widgets_wrapper())

In [15]:
#for woody
v_detailed_2 = transfer_details(v_smooth_2,di_B_2,j_longest_B_2)
p_3 = mp.plot(v_detailed_2, f, c=labels, shading = {"wireframe": True, "flat": False})
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4, 5), value=1), FloatSlider(value=0.0, desc…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

# Performance

In [16]:
from scipy.linalg import cho_factor, cho_solve
c, lower = cho_factor(Aff.todense(), lower=True)
xc = handle_vertex_positions[labels>0, :]
x = cho_solve((c, lower), -Afc @ xc)
v_smooth_fast = handle_vertex_positions.copy() #B
v_smooth_fast[labels==0] = x
p = mp.plot(v_smooth_fast, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

In [17]:
#B'
xc_2 = handle_vertex_positions[labels>0, :]
x_2 = cho_solve((c, lower), -Afc @ xc)
v_smooth_2_fast = handle_vertex_positions
v_smooth_2_fast[labels==0] = x_2
# p_2_fast = mp.plot(v_smooth_2, f, c=labels, shading = {"wireframe": True, "flat": False})
p_2_fast = mp.plot(v_smooth_2_fast, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

In [18]:
#caculate for mesh B
di_B_fast, j_longest_B_fast = construct_mesh_details(v_smooth_fast)
#caculate for mesh B'
di_B_2_fast, j_longest_B_2_fast = construct_mesh_details(v_smooth_2_fast)

In [19]:
# edges = v_smooth_2_fast[j_longest_B_2_fast] - v_smooth_2_fast
# ni = igl.per_vertex_normals(v_smooth_2_fast, f)
# print(edges.shape,ni.shape)
# print(np.einsum('ij,ij->i', edges, ni).reshape(-1, 1).shape)
# einsum is new, and is presumably trying to be better about cache alignment and other memory access issues, 
# while many of the older numpy functions focus on a easily portable implementation over a heavily optimized one. 

In [20]:
def transfer_details_fast(v_sm, di, j_longest):
    di_new = np.zeros_like(di)
    v_detailed = v_sm.copy()
    ni = igl.per_vertex_normals(v_sm, f)
    edges = v_sm[j_longest] - v_sm
    xi = np.zeros_like(ni)
#     xi = edges - ni*(np.sum(edges * ni, axis=1).reshape(-1, 1))
    xi = edges - ni*(np.einsum('ij,ij->i', edges, ni).reshape(-1, 1))
    xi /= np.linalg.norm(xi).reshape(-1, 1)
    yi = np.cross(ni, xi)
    for i in range(v.shape[0]):
#         di_new[i] = np.array([di[i] @ xi, di[i] @ yi, di[i] @ ni])
        di_new[i] = np.einsum('i,ij',di[i],np.vstack((xi[i], yi[i], ni[i])))
    v_detailed += di_new    
    return v_detailed

In [21]:
v_detailed_2_fast = transfer_details_fast(v_smooth_2_fast,di_B_2_fast,j_longest_B_2_fast)

In [22]:
p = mp.plot(v_detailed_2_fast, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4, 5), value=1), FloatSlider(value=0.0, desc…

<function __main__.pos_f(s, x, y, z, α, β, γ)>